<a href="https://colab.research.google.com/github/jaimegonzalezc/KC-NLP-Practica/blob/main/PracticaNLP-2-Preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica Natural Language Processing - KeepCoding

## 2. Preprocesado del texto
Ahora vamos a procesar el texto para poder pasar a la etapa de entrenamiento con los datos lo más limpios posibles.

Se va a realizar un conjunto de funciones que realicen una modificación en el texto introducido para después agruparlas en una función unica de procesado para pasar por los modelos de análisis de sentimiento.

Por ello, cargamos librerías y vamos función a función realizando fases del procesado:

In [1]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

data = pd.read_json('/content/drive/MyDrive/Bootcamp IA/NLP/Práctica/Digital_Music_5.json', lines=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


En primer lugar se van a realizar las funciones que no requieren de tokenización, para una vez limpiado el texto, tokenizarlo y proceder con las siguientes fases del procesado.

### Signos de puntuación

In [2]:
def eliminar_signos_puntuacion(texto):
    return texto.translate(str.maketrans('', '', string.punctuation))

texto = "Hi, world!"
print(eliminar_signos_puntuacion(texto))

Hi world


### Eliminación de números
En las reviews podrían encontrarse números que no aportan nada en el análisis de sentimiento y pueden dar problemas al vectorizar las palabras.

In [3]:
def eliminar_numeros(texto):
    return re.sub(r'\d+', '', texto)

texto = "There are 3 balls in this bag."
print(eliminar_numeros(texto))

There are  balls in this bag.


### Eliminación de URLs
Tras revisar los datos, se ha encontrado que en algunos comentarios la gente deja URLs para apoyar su valoración, por lo que también se debe eliminar utilizando para ello regex.

In [4]:
def eliminar_urls_menciones(texto):
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\@\w+|\#','', texto)
    return texto

texto = "Great product! It works perfectly and arrived on time. Check it out here: https://www.amazon.com/dp/B07QK2SPP7."
print(eliminar_urls_menciones(texto))

Great product! It works perfectly and arrived on time. Check it out here: 


### Borrado de Stopwords
Para eliminar los Stopwords, ya debemos introducir la información tokenizada, por lo que antes de comprobar la función tokenizaremos el texto de entrada.

In [6]:
def eliminar_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

texto = "This is an example sentence with some stopwords in it"
tokens = word_tokenize(texto)
print(eliminar_stopwords(tokens))

['This', 'example', 'sentence', 'stopwords']


### Lematizado de tokens
Lematizamos las palabras para extraer el lema de cada palabra y agrupar palabras de la misma familia.

In [7]:
def lematizar_palabras(tokens):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) for word in tokens]

texto = "The children are playing in the parks and they will be running tomorrow"
tokens = word_tokenize(texto)
print(lematizar_palabras(tokens))

['The', 'child', 'are', 'playing', 'in', 'the', 'park', 'and', 'they', 'will', 'be', 'running', 'tomorrow']


### Función general
Por último, vamos a generar esa función final y a probarla con nuestro dataset.

In [9]:
def preprocesar_texto(texto):
    texto = texto.lower() # Convertir a minúsculas
    texto = eliminar_signos_puntuacion(texto)
    texto = eliminar_numeros(texto)
    texto = eliminar_urls_menciones(texto)
    tokens = word_tokenize(texto)
    tokens = eliminar_stopwords(tokens)
    tokens = lematizar_palabras(tokens)
    texto_procesado = ' '.join(tokens) # Volvemos a unir los tokens en texto
    return texto_procesado

# Aplicar la función de preprocesamiento a la columna de texto
data['reviewText_procesado'] = data['reviewText'].apply(preprocesar_texto)

# Mostrar las primeras filas del DataFrame procesado para comparar
print(data[['reviewText', 'reviewText_procesado']].head())


                                          reviewText  \
0  It's hard to believe "Memory of Trees" came ou...   
1  A clasically-styled and introverted album, Mem...   
2  I never thought Enya would reach the sublime h...   
3  This is the third review of an irish album I w...   
4  Enya, despite being a successful recording art...   

                                reviewText_procesado  
0  hard believe memory tree came year agoit held ...  
1  clasicallystyled introverted album memory tree...  
2  never thought enya would reach sublime height ...  
3  third review irish album write today others cr...  
4  enya despite successful recording artist doesn...  


Para acabar, vamos a guardar los datos procesados para trabajar con ellos en el entramiento de los modelos:

In [10]:
data.to_csv('/content/drive/MyDrive/Bootcamp IA/NLP/Práctica/reviews_preprocesadas.csv', index=False)